In [1]:
from data_loader import data_importing
training = r'C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\train'
testing = r'C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\test'

# This Function inputs the training images and masks directory,
# and test images and rezises the images and returns the numpy arrays that are
# resized from 101 x 101 to 128 x 128.
X_train, Y_train = data_importing(path_train=training, path_test=testing)

Using TensorFlow backend.


Getting and resizing train images and masks ... 



Done!


In [2]:
from image_augmentor import aug
train_image_path = r"C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\train\images"
train_mask_path = r"C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\train\masks"
path_train = r'C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\train'
num_images_augmented = 1000


Aug_X_train, Aug_Y_train = aug(train_image_path=train_image_path,
                               train_mask_path=train_mask_path,
                               path_train=path_train,
                              num_images_augmented=num_images_augmented)

Initialised with 4000 image(s) found.
Output directory set to C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\train\images\output.

4000 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGB size=101x101 at 0x18FB5668>: 100%|██████████| 1000/1000 [00:04<00:00, 201.73 Samples/s]


Getting and resizing train images and masks ... 



Done!


In [3]:
import numpy as np
X_train = np.concatenate((X_train, Aug_X_train), axis=0)
Y_train = np.concatenate((Y_train, Aug_Y_train), axis=0)

# Let's use the first 350 images from our trainset for our test sample.
X_test_out_of_train = X_train[:350]
y_test_out_of_train = Y_train[:350]


X_train_w_aug = X_train[350:]
y_train_w_aug = Y_train[350:]

In [4]:
from Modeling import UNET_MODEL

In [5]:
results = UNET_MODEL(X_train_w_aug=X_train_w_aug,
           y_train_w_aug=y_train_w_aug,
          X_test_out_of_train = X_test_out_of_train,
          y_test_out_of_train=y_test_out_of_train)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 8)  80          lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 8)  584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [6]:
from data_loader import data_importing_test

In [8]:
# Testing Images
X_test, sizes_test, test_ids = data_importing_test(path_train = r'C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\train',
                                                  path_test = r'C:\Users\uschlac\Documents\Practicum\Practicum_2\Salt_Deposit_Identification-master\Data\test')

Getting and resizing test images ... 



Done!


In [9]:
from keras.models import load_model
from mean_iou import mean_iou
import tqdm
from skimage.transform import resize

# Predict on train, val and test
model = load_model('model-tgs-salt-1.h5', custom_objects={'mean_iou': mean_iou})
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in tqdm.tnrange(len(preds_test)):
    # We need to resize the images from 128 x 128 back to 101 x 101 pixels. 
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

18000/18000 [==============================] - ETA: 6: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 58s - ETA: 56 - ETA: 55 - ETA: 53 - ETA: 52 - ETA: 51 - ETA: 50 - ETA: 49 - ETA: 48 - ETA: 48 - ETA: 47 - ETA: 46 - ETA: 46 - ETA: 45 - ETA: 45 - ETA: 44 - ETA: 44 - ETA: 43 - ETA: 43 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 31 - ET

In [10]:
from submission import RLenc
from submission import get_submission
from tqdm import tqdm_notebook as tqdm_NB

pred_dict = {fn[:-4]:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_NB(enumerate(test_ids))}
get_submission(pred_dict)